<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
import numpy as np

In [8]:
# SSE(Sum of Squares for Error, 오차제곱합) 손실함수 구현
def sse(y, t):
    loss = np.sum((y - t) ** 2)
    return loss
    

# 정답에 근사했을 경우
y = np.array([0.1, 0.05, 0.6, 0., 0.05, 0.1, 0., 0.1, 0., 0,])
t = np.array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0])

loss = sse(y, t)
print(loss)

# 정답과 멀을 경우
y = np.array([0.1, 0.05, 0.6, 0., 0.05, 0.1, 0., 0.1, 0., 0,])
t = np.array([0, 0, 0, 0, 0, 0, 1, 0, 0, 0])

loss = sse(y, t)
print(loss)

0.19500000000000006
1.395


In [9]:
def cross_entropy(y, t):
    delta = 1e-7
    loss = -np.sum(t * np.log(y + delta))
    return loss

# 정답에 근사했을 경우
y = np.array([0.1, 0.05, 0.6, 0., 0.05, 0.1, 0., 0.1, 0., 0,])
t = np.array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0])

loss = cross_entropy(y, t)
print(loss)

# 정답과 멀을 경우
y = np.array([0.1, 0.05, 0.6, 0., 0.05, 0.1, 0., 0.1, 0., 0,])
t = np.array([0, 0, 0, 0, 0, 0, 1, 0, 0, 0])

loss = cross_entropy(y, t)
print(loss)

0.510825457099338
16.118095650958317


In [19]:
a = np.array([[1,2,3], [4,5,6], [7,8,9]])
a.size

9

In [21]:
from dataset.mnist import load_mnist

(X_train, y_train), (X_test, y_test) = load_mnist(normalize=True, one_hot_label=True)
print(X_train.shape, y_train.shape)


(60000, 784) (60000, 10)


In [26]:
# 배치 사이즈만큼의 배치 데이터 추출
train_size = X_train.shape[0]
batch_size = 10
batch_index = np.random.choice(train_size, batch_size)

X_train_batch = X_train[batch_index]
y_train_batch = y_train[batch_index]
print(X_train_batch.shape, y_train_batch.shape)

(10, 784) (10, 10)


In [27]:
# 레이블이 원-핫 인코딩일 경우 CEE 손실함수
def cross_entropy_ohe(y, t):
    # 데이터가 1개만 들어왔을 경우(1차원) -> 2차원으로 변환
    if y.ndim == 1:
        y = y.reshape(1, y.size)
        t = t.reshape(1, t.size)
    
    batch_size = y.shape[0]
    loss = -np.sum(t * np.log(y + 1e-7))
    return loss

In [49]:
from dataset.mnist import load_mnist
import pickle

(X_train, y_train), (X_test, y_test) = load_mnist(normalize=True, one_hot_label=False)

def init_network():
    with open('/Users/younghun/Desktop/gitrepo/deep-learning-from-scratch/ch03/sample_weight.pkl', 'rb') as f:
        network = pickle.load(f)
    return network

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def softmax(x):
    max_x = np.max(x)
    exp_x = np.exp(x - max_x)
    exp_x_sum = np.sum(exp_x)
    return exp_x / exp_x_sum

def forward(network, x):
    w1, w2, w3 = network['W1'], network['W2'], network['W3']
    b1, b2, b3 = network['b1'], network['b2'], network['b3']
    
    a1 = np.matmul(x, w1) + b1
    z1 = sigmoid(a1)
    a2 = np.matmul(z1, w2) + b2
    z2 = sigmoid(a2)
    a3 = np.matmul(z2, w3) + b3
    y = softmax(a3)
    
    return y

batch_size = 10
batch_idx = np.random.choice(X_train.shape[0], batch_size)
X_batch = X_train[batch_idx]
y_batch = y_train[batch_idx]

network = init_network()
y_pred = forward(network, X_batch)

def cross_entropy(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
        
    batch_size = y.shape[0]
    #print(y.shape, t.shape)                            # y는 2차원(원-핫인코딩) t는 1차원(레이블인코딩)
    #이해못했던 부분!)=> print(y[np.arange(batch_size), t]) # y값의 열 인덱스 번호 = t의 레이블 값임!
    loss = -np.sum(np.log(y[np.arange(batch_size), t] + 1e-7)) / batch_size
    return loss

loss = cross_entropy(y_pred, y_batch)
print(loss)

2.8765232086181642
